# Circuit: schematic creation and analysis

This example shows how you can use PyAEDT to create a circuit design and
run a Nexxim time-domain simulation.


# Perform required imports

Perform required imports.


In [ ]:
# sphinx_gallery_thumbnail_path = 'Resources/circuit.png'

import ansys.aedt.core

# Set AEDT version

Set AEDT version.


In [ ]:
aedt_version = "2024.2"

# Set non-graphical mode

Set non-graphical mode. You can set `non_graphical` either to `True` or
`False`. The Boolean parameter `new_thread` defines whether to create a
new instance of AEDT or try to connect to an existing instance of it.


In [ ]:
non_graphical = False
new_thread = True

# Launch AEDT and Circuit

Launch AEDT and Circuit. The `ansys.aedt.core.Desktop`{.interpreted-text
role="class"} class initializes AEDT and starts the specified version in
the specified mode.


In [ ]:
desktop = ansys.aedt.core.launch_desktop(aedt_version, non_graphical, new_thread)
aedt_app = ansys.aedt.core.Circuit(project=ansys.aedt.core.generate_unique_project_name())
aedt_app.modeler.schematic.schematic_units = "mil"

# Create circuit setup

Create and customize an LNA (linear network analysis) setup.


In [ ]:
setup1 = aedt_app.create_setup("MyLNA")
setup1.props["SweepDefinition"]["Data"] = "LINC 0GHz 4GHz 10001"

# Create components

Create components, such as an inductor, resistor, and capacitor.


In [ ]:
inductor = aedt_app.modeler.schematic.create_inductor(name="L1", value=1e-9, location=[0, 0])
resistor = aedt_app.modeler.schematic.create_resistor(name="R1", value=50, location=[500, 0])
capacitor = aedt_app.modeler.schematic.create_capacitor(name="C1", value=1e-12, location=[1000, 0])

# Get all pins

Get all pins of a specified component.


In [ ]:
pins_resistor = resistor.pins

# Create port and ground

Create a port and a ground, which are needed for the circuit analysis.


In [ ]:
port = aedt_app.modeler.components.create_interface_port(name="myport", location=[-200, 0] )
gnd = aedt_app.modeler.components.create_gnd(location=[1200, -100])

# Connect components

Connect components with wires.


In [ ]:
port.pins[0].connect_to_component(assignment=inductor.pins[0], use_wire=True)
inductor.pins[1].connect_to_component(assignment=resistor.pins[1], use_wire=True)
resistor.pins[0].connect_to_component(assignment=capacitor.pins[0], use_wire=True)
capacitor.pins[1].connect_to_component(assignment=gnd.pins[0], use_wire=True)

# Create transient setup

Create a transient setup.


In [ ]:
setup2 = aedt_app.create_setup(name="MyTransient", setup_type=aedt_app.SETUPS.NexximTransient)
setup2.props["TransientData"] = ["0.01ns", "200ns"]
setup3 = aedt_app.create_setup(name="MyDC", setup_type=aedt_app.SETUPS.NexximDC)

# Solve transient setup

Solve the transient setup.


In [ ]:
aedt_app.analyze_setup("MyLNA")
aedt_app.export_fullwave_spice()

# Create report

Create a report that plots solution data.


In [ ]:
solutions = aedt_app.post.get_solution_data(expressions=aedt_app.get_traces_for_plot(category="S"))
solutions.enable_pandas_output = True
real, imag = solutions.full_matrix_real_imag
print(real)

# Plot data

Create a plot based on solution data.


In [ ]:
fig = solutions.plot()

# Close AEDT

After the simulation completes, you can close AEDT or release it using
the `ansys.aedt.core.Desktop.force_close_desktop`{.interpreted-text
role="func"} method. All methods provide for saving the project before
closing.


In [ ]:
desktop.release_desktop()